# Support Vector Machines with Diabetes Data

Adapted from Lesmeister (2015) Chapter 5

The data that we will examine was originally collected by the National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK). It consists of 532 observations and eight input features along with a binary outcome (Yes/No). The patients in this study were of Pima Indian descent from South Central Arizona. The NIDDK data shows that since the past 30 years, research has helped scientists to prove that obesity is a major risk factor in the development of diabetes. The Pima Indians were selected for the study as one-half of the adult Pima Indians have diabetes and 95 percent of those with diabetes are overweight. The analysis will focus on adult women only. Diabetes was diagnosed according to the WHO criteria and was of the type of diabetes that is known as type 2. In this type of diabetes, the pancreas is still able to function and produce insulin and it used to be referred to as non-insulin-dependent diabetes.

Our task is to examine and predict those individuals that have diabetes or the risk factors that could lead to diabetes in this population.

## Load libraries and data

In [ ]:
library(data.table) # to handle the data in a more convenient manner
library(tidyverse) # for a better work flow and more tools to wrangle and visualize the data
library(plotly) # for interactive visualizations
library(e1071) # for SVM
library(caret) # for model tuning and confusion matrix
library(MASS) # for data
library(kernlab) # for SVM feature selection
library(BBmisc) # data normalization
library(corrplot) # correlation plot

In [ ]:
data(Pima.tr, package = "MASS")
data(Pima.te, package = "MASS")

There are two separate dataset for train and test

In [ ]:
pima_tr_dt <- as.data.table(Pima.tr)
pima_te_dt <- as.data.table(Pima.te)

## Explore dataset

In [ ]:
str(pima_tr_dt)

In [ ]:
?Pima.tr

```
These data frames contains the following columns:

npreg
number of pregnancies.

glu
plasma glucose concentration in an oral glucose tolerance test.

bp
diastolic blood pressure (mm Hg).

skin
triceps skin fold thickness (mm).

bmi
body mass index (weight in kg/(height in m)\^2).

ped
diabetes pedigree function.

age
age in years.

type
Yes or No, for diabetic according to WHO criteria.
```

The datasets are contained in the R package, MASS. One data frame is named Pima.tr and the other is named Pima.te. Instead of using these as separate train and test sets, we will combine them and create our own in order to discover how to do such a task in R.

In [ ]:
pima <- rbind(pima_tr_dt, pima_te_dt)

In [ ]:
str(pima)

In [ ]:
tibble::glimpse(pima)

In [ ]:
pima_melt <- melt(pima, id.var = "type")

In [ ]:
pima_melt

In [ ]:
pima %>%
data.table::melt(id.var = "type") %>%
ggplot(aes(x = type, y = value)) +
geom_boxplot() +
facet_wrap(~variable, ncol = 2)

This is an interesting plot because it is difficult to discern any dramatic differences in the plots, probably with the exception of glucose (glu). As you may have suspected, the fasting glucose appears to be significantly higher in the patients currently diagnosed with diabetes. The main problem here is that the plots are all on the same y axis scale. We can fix this and produce a more meaningful plot by standardizing  the values and then re-plotting.

In [ ]:
pima_scale <- pima[,cbind(BBmisc::normalize(.SD), type), .SDcols = !"type"]
pima_scale

In [ ]:
pima_scale %>%
data.table::melt(id.var = "type") %>%
ggplot(aes(x = type, y = value)) +
geom_boxplot() +
facet_wrap(~variable, ncol = 2)

With the features scaled, the plot is easier to read. In addition to glucose, it appear that the other features may differ by type, in particular, age.

Before splitting this into train and test sets, let's have a look at the correlation with the R function, cor(). This will produce a matrix instead of a plot of the Pearson correlations:

In [ ]:
cor(pima_scale[,!"type"]) %>%

corrplot::corrplot.mixed(upper = "ellipse",
                         lower = "number",
                         tl.pos = "lt",
                         number.cex = .5,
                         lower.col = "black",
                         tl.cex = 0.7)

There are a couple of correlations to point out, npreg/age and skin/bmi.  

Multi-collinearity is generally not a problem with these methods, assuming that they are properly trained and the hyperparameters are tuned.

In [ ]:
table(pima_scale$type)

## Partitioning the data

In [ ]:
set.seed(502)

In [ ]:
train <- pima_scale[,sample(.I, 0.7 * .N)]

In [ ]:
pima_train <- pima_scale[train]
pima_test <- pima_scale[-train]

## Build and train a model

### Linear model

We will use the e1071 package to build our SVM models. We will start with a linear support vector classifier and then move on to the nonlinear versions. The e1071 package has a nice function for SVM called tune.svm(), which assists in the selection of the tuning parameters/kernel functions.

The tune.svm() function from the package uses cross-validation to optimize the tuning parameters. Let's create an object called linear.tune and call it using the summary() function, as follows:

In [ ]:
linear_tune <- e1071::tune.svm(type ~ .,
                               data = pima_train,
                              kernel = "linear",
                              cost = c(0.001, 0.01, 0.1, 1, 5, 10))

What is "cost"?


https://www.quora.com/What-is-the-intuition-behind-the-Cost-and-Gamma-parameters-in-SVM
> C is how much we penalize the SVM for data points within the margin (where "within" can also mean on the wrong side of the dividing hyperplane). Large C means penalize a lot. If your C is large, the SVM will try to find a hyperplane and margin so that there are few very points within the margin, which could mean an overly complex model with a small margin if the points aren't easily separable. A lower C gives higher error on the training set, but finds a larger margin that might be more robust.


In [ ]:
summary(linear_tune)

The optimal cost function is one for this data and leads to a misclassification error of roughly 21 percent. We can make predictions on the test data and examine that as  well using the predict() function and applying newdata=test:

In [ ]:
best_linear <- linear_tune$best.model

In [ ]:
tune_test <- predict(best_linear, newdata = pima_test)

In [ ]:
caret::confusionMatrix(tune_test, pima_test$type, positive = "Yes")

### Non-linear model

We will now see if non-linear methods will improve our performance and also use cross-validation to select tuning parameters.

#### Polynomial

The first kernel function that we will try is polynomial, and we will be tuning two parameters: a degree of polynomial (degree) and kernel coefficient (coef0). The polynomial order will be 3, 4, and 5 and the coefficient will be in increments from 0.1 to 4, as follows:

In [ ]:
set.seed(123)
poly_tune <- e1071::tune.svm(type ~ .,
                            data = pima_train,
                            kernel = "polynomial",
                            degree = 3:5,
                            coef0 = c(0.1, 0.5, 1:4))

In [ ]:
summary(poly_tune)

The model has selected degree of 3 for the polynomial and coefficient of 1. 

Just as the linear SVM, we can create predictions on the test set with these parameters, as follows:

In [ ]:
best_poly <- poly_tune$best.model

In [ ]:
poly_test <- predict(best_poly, newdata = pima_test)

In [ ]:
caret::confusionMatrix(poly_test, pima_test$type, positive = "Yes")

This did not perform quite as well as the linear model.

#### Radial

We will now run the radial basis function. In this instance, the one parameter that we will solve for is gamma, which we will examine in increments of 0.1 to 4. If gamma is too small, the model will not capture the complexity of the decision boundary; if it is too large, the model will severely overfit:

In [ ]:
set.seed(123)
rbf_tune <- e1071::tune.svm(type ~ .,
                           data = pima_train,
                           kernel = "radial",
                           gamma = c(0.1, 0.5, 1:4))

https://www.quora.com/What-is-the-intuition-behind-the-Cost-and-Gamma-parameters-in-SVM

>Gamma is not technically an SVM hyperparameter. It is a parameter of the kernel. In the RBF kernel, it adjusts how similar two points must be to be considered "similar". You can think of the RBF kernel as constructing a hypersphere around each point, whose density is described by a Gaussian function. The value that the kernel generates for points (a, b) is the density of a's hypersphere at point b. Gamma controls the standard deviation of the Gaussian function. Larger gamma means smaller standard deviation around each point.

In [ ]:
summary(rbf_tune)

The best gamma value is 0.1 and the performance at this setting does not seem to improve much over the other SVM models. We will check for the test set as well in the following way:

In [ ]:
best_rbf <- rbf_tune$best.model

In [ ]:
rbf_test <- predict(best_rbf, newdata = pima_test)

In [ ]:
caret::confusionMatrix(rbf_test, pima_test$type, positive = "Yes")

The performance is downright abysmal.

#### Sigmoid

One last shot to improve here would be with kernel="sigmoid". We will be solving for two parameters that are gamma and the kernel coefficient (coef0)

In [ ]:
set.seed(123)
sigmoid_tune <- e1071::tune.svm(type ~ .,
                               data = pima_train,
                               kernel = "sigmoid",
                               gamma = c(0.1, 0.5, 1:4),
                               coef0 = c(0.1, 0.5, 1:4))

In [ ]:
summary(sigmoid_tune)

This error rate is in line with the linear model. It is now just a matter of whether it performs better on the test set or not:

In [ ]:
best_sigmoid <- sigmoid_tune$best.model

In [ ]:
sigmoid_test <- predict(best_sigmoid, newdata = pima_test)

In [ ]:
caret::confusionMatrix(sigmoid_test, pima_test$type, positive = "Yes")

Lo and behold! We finally have a test performance that is in line with the performance on the train data. It appears that we can choose the sigmoid  kernel as the best predictor.

So far we played around with different models. Now, let's evaluate their performance along with the linear model using metrics other than just the accuracy.

The function produces some items that we already covered such as Accuracy and 
Kappa. Here are the other stats that it produces:

- No Information Rate is the proportion of the largest class—63 percent did not have diabetes.
- P-Value is used to test the hypothesis that the accuracy is actually better than No Information Rate.
- We will not concern ourselves with Mcnemar's Test, which is used for the analysis of the matched pairs, primarily in epidemiology studies
- Sensitivity is the true positive rate; in this case, the rate of those not having diabetes has been correctly identified as such.
- Specificity is the true negative rate or, for our purposes, the rate of a diabetic that has been correctly identified.
- The positive predictive value (Pos Pred Value) is the probability of someone in the population classified as being diabetic and truly has the disease. The following formula is used:
- The negative predictive value (Neg Pred Value) is the probability of someone in the population classified as not being diabetic and truly does not have the disease. The formula for this is as follows:
- Prevalence is the estimated population prevalence of the disease, calculated here as the total of the second column (the Yes column) divided by the  total observations.
- Detection Rate is the rate of the true positives that have been identified—in our case, 35—divided by the total observations.
- Detection Prevalence is the predicted prevalence rate, or in our case, the bottom row divided by the total observations.
- Balanced Accuracy is the average accuracy obtained from either class.  This measure accounts for a potential bias in the classifier algorithm,  thus potentially overpredicting the most frequent class. This is simply Sensitivity + Specificity divided by 2.

## Feature selection

All is not lost on feature selection and we want to take some space to show you a quick way in how to begin exploring this matter. It will require some trial and error on your part.

Again, the caret package helps out in this matter as it will run a cross-validation on a linear SVM based on the kernlab package.

To do this, we will need to set the random seed, specify the cross-validation method in the caret's rfeControl() function, perform a recursive feature selection with the rfe() function, and then test how the model performs on the test set.

In rfeControl(), you will need to specify the function based on the model being used. There are several different functions that you can use. Here we will need lrFuncs. To see a list of the available functions, your best bet is to explore the documentation with ?rfeControl and ?caretFuncs. The code for this example is as follows:

In [ ]:
set.seed(123)
rfeCNTL <- caret::rfeControl(functions = lrFuncs,
                            method = "cv",
                            number = 10)

In [ ]:
svm_features <- caret::rfe(pima_train[,1:7],
                           pima_train[,type],
                          sizes = 7:4,
                          rfeControl = rfeCNTL,
                          method = "svmLinear")

To create the svm_features object, it was important to specify the inputs and response factor, number of input features via sizes, and linear method from kernlab, which is the svmLinear syntax. Other options are available using this method, such as svmPoly. No method for a sigmoid kernel is available. Calling the object allows us to see how the various feature sizes perform, as follows:

In [ ]:
svm_features

Counter-intuitive as it is, the four variables perform quite well by themselves as well as when skin and bp are included.

Let's try this out on the test set, remembering that the accuracy in the full model was 78 percent:

In [ ]:
formula5 <- reformulate(svm_features$optVariables, "type")

In [ ]:
svm_5 <- e1071::svm(formula5,
                   data = pima_train,
                   kernel = "linear")

In [ ]:
svm_5_predict <- predict(svm_5,
                        newdata = pima_test)

In [ ]:
caret::confusionMatrix(svm_5_predict, pima_test$type, positive = "Yes")

This did not perform as well and we can stick with the full model. You can see through trial and error how this technique can play in order to determine some simple identification of feature importance.